In [1]:
import polars as pl
from record_consolidation.graphs import (
    unconsolidated_df_to_subgraphs,
)
from record_consolidation.subgraph_post_processing.specific_algs.deprecated__split_high_betweenness import (
    draw_graph,
)
import networkx as nx

from record_consolidation.subgraph_post_processing.specific_algs.partition_companies import (
    partition_companies_graph_where_necessary,
)

from record_consolidation.subgraph_post_processing.specific_algs.partitioning_algs import (
    partition_via_louvain,
)

from functools import partial
from record_consolidation.graphs import extract_normalized_atomic

from record_consolidation.df_consolidations import (
    normalize_subset,
)

/Users/mgaster/repos/record-consolidation/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from typing import Iterable
from warnings import warn


def extract_specific_name_subgraph(
    connected_subgs: Iterable[nx.Graph], name: str
) -> nx.Graph:
    collected: list[nx.Graph] = []
    for subg in connected_subgs:
        for n in subg.nodes.data():
            if name.lower() in (n[0]).lower():
                collected.append(subg)
    collected = set(collected)
    if len(collected) == 0:
        raise ValueError(f"Could not find {name=} in connected_subgs.")
    if len(collected) > 1:
        alert_str = f"{len(collected)} subgraphs have {name=}.\n{[len(subg.nodes) for subg in collected]=}"
        warn(alert_str)
        return max(collected, key=lambda g: len(g.nodes))

        # raise ValueError(alert_str)
    return tuple(collected)[0]

In [3]:
# votes: pl.DataFrame = access_db_table("raw_output", "votes")
votes: pl.DataFrame = pl.read_parquet("raw_votes.parquet")
COMPANY_COLS = pl.col(["issuer_name", "cusip", "isin", "figi"])
MEETING_COLS = pl.col(["issuer_name", "meeting_date", "cusip", "isin", "figi"])

In [4]:
normed5 = normalize_subset(
    votes,
    connected_subgraphs_postprocessor=partition_companies_graph_where_necessary,
    cols_to_normalize=["issuer_name", "cusip", "isin", "figi"],
)

Post-processing subgraphs.


7615it [00:02, 2954.06it/s]


new_null_count=524_700, og_null_count=997_930


/Users/mgaster/repos/record-consolidation/src/record_consolidation/df_consolidations.py:61: UserWarning: New nulls added -- detected in cusip.
  warn(warning_str)
/Users/mgaster/repos/record-consolidation/src/record_consolidation/df_consolidations.py:61: UserWarning: New nulls added -- detected in isin.
  warn(warning_str)
/Users/mgaster/repos/record-consolidation/src/record_consolidation/df_consolidations.py:61: UserWarning: New nulls added -- detected in figi.
  warn(warning_str)


In [6]:
votes.select(COMPANY_COLS.is_null().sum()) / normed5.select(pl.all().is_null().sum())

issuer_name,cusip,isin,figi
f64,f64,f64,f64
NaN,4.393469,43.37155,1.516752


In [7]:
votes["issuer_name"].n_unique()

19277

In [7]:
normed5.select(COMPANY_COLS.is_null().mean()) * 100

issuer_name,cusip,isin,figi
f64,f64,f64,f64
0.0,1.101106,0.544327,65.71525


In [8]:
normed5.select(COMPANY_COLS.is_null().mean()) * 100

issuer_name,cusip,isin,figi
f64,f64,f64,f64
0.0,1.100849,0.544198,65.715379


In [9]:
normed5["issuer_name"].value_counts(sort=True).head(30).to_series().to_list()

['MICROSOFT CORPORATION',
 'THE WALT DISNEY COMPANY',
 'AMAZON.COM, INC.',
 'JPMORGAN CHASE & CO.',
 'THE PROCTER & GAMBLE COMPANY',
 'ORACLE CORPORATION',
 'CISCO SYSTEMS, INC.',
 'APPLE INC.',
 'JOHNSON & JOHNSON',
 'PEPSICO, INC.',
 'VISA INC.',
 'ALPHABET INC.',
 'MASTERCARD INCORPORATED',
 'ABBVIE INC.',
 'MERCK & CO., INC.',
 'CHUBB LIMITED',
 'THE HOME DEPOT, INC.',
 'CHEVRON CORPORATION',
 'NVIDIA CORPORATION',
 'TE CONNECTIVITY LTD.',
 'EXXON MOBIL CORPORATION',
 'BANK OF AMERICA CORPORATION',
 'META PLATFORMS, INC.',
 'UNITEDHEALTH GROUP INCORPORATED',
 'VERIZON COMMUNICATIONS INC.',
 'SALESFORCE, INC.',
 'THE COCA-COLA COMPANY',
 "MCDONALD'S CORPORATION",
 'WELLS FARGO & COMPANY',
 'ADOBE INC.']

In [10]:
for name in (
    # normed5["issuer_name"].value_counts(sort=True).head(30).to_series().to_list()
    [
        "microsoft",
        "disney",
        "amazon",
        "jpmorgan",
        "procter",
        "oracle",
        "apple",
        "visa",
        "exxon",
        "chevron",
        "meta platform",
        "air transport",
        "bank of america",
        "blackrock",
        "suncor",
        "conoco phillips",
        "phillips 66",
        "british petroleum",
    ]
):
    print("*" * 60, name.upper(), "*" * 60)
    t_unnormalized = (
        votes.select(COMPANY_COLS)
        .filter(pl.col("issuer_name").str.contains(f"(?i){name}"))
        .group_by("issuer_name")
        .agg(pl.len(), pl.all().unique())
        .with_columns((pl.col("len") * 100 / pl.col("len").sum()).alias("% of obs"))
        .sort("len", descending=True)
    )
    t = (
        normed5.select(COMPANY_COLS)
        .filter(pl.col("issuer_name").str.contains(f"(?i){name}"))
        .group_by("issuer_name")
        .agg(pl.len(), pl.all().unique())
        .with_columns((pl.col("len") * 100 / pl.col("len").sum()).alias("% of obs"))
        .sort("len", descending=True)
    )
    print("Un-Normalized")
    display(t_unnormalized)
    print("Normalized")
    display(t)

************************************************************ MICROSOFT ************************************************************
Un-Normalized


issuer_name,len,cusip,isin,figi,% of obs
str,u32,list[str],list[str],list[str],f64
"""MICROSOFT CORPORATION""",2555,"[""594918105"", null, ""594918104""]","[null, ""US5949181045""]",[null],54.028336
"""Microsoft Corporation""",1795,"[""594918103"", ""000594918"", … null]","[""US5949181045"", null]","[null, ""BBG000BPH459""]",37.957285
"""Microsoft Corp""",226,"[""594918104"", null]","[null, ""US5949181045""]","[""BBG000BPH459"", null]",4.779023
"""MICROSOFT CORPORATION""",64,"[""594918104""]","[null, ""US5949181045""]",[null],1.353352
"""Microsoft Corp.""",60,"[""594918104""]","[""US5949181045"", null]","[null, ""BBG000BPH459""]",1.268767
…,…,…,…,…,…
"""MICROSOFT""",2,"[""594918104"", null]",[null],[null],0.042292
"""Microsoft Corporation (MSFT)""",2,"[""594918104""]","[""US5949181045""]",[null],0.042292
"""MICROSOFT CORPORATION COM""",2,"[""594918104""]","[""US5949181045""]",[null],0.042292


Normalized


issuer_name,len,cusip,isin,figi,% of obs
str,u32,list[str],list[str],list[str],f64
"""MICROSOFT CORPORATION""",4729,"[""594918104""]","[""US5949181045""]","[""BBG000BPH459""]",99.915487
"""MICROSOFT""",2,[null],[null],[null],0.042256
"""MICROSOFT CORP""",2,[null],[null],[null],0.042256


************************************************************ DISNEY ************************************************************
Un-Normalized


...

issuer_name,len,cusip,isin,figi,% of obs
str,u32,list[str],list[str],list[str],f64
"""PHILLIPS 66""",722,"[""718546104""]","[""US7185461040""]","[""BBG00286S4N9""]",100.0


************************************************************ BRITISH PETROLEUM ************************************************************
Un-Normalized


issuer_name,len,cusip,isin,figi,% of obs
str,u32,list[str],list[str],list[str],f64


Normalized


issuer_name,len,cusip,isin,figi,% of obs
str,u32,list[str],list[str],list[str],f64


In [50]:
votes.filter(normed5["issuer_name"] != votes["issuer_name"]).select(COMPANY_COLS)

issuer_name,cusip,isin,figi
str,str,str,str
"""AMPHENOL CORPORATION""","""032095101""","""US0320951017""",null
"""THE SHERWIN-WILLIAMS COMPANY""","""824348106""","""US8243481061""",null
"""THE SHERWIN-WILLIAMS COMPANY""","""824348106""","""US8243481061""",null
"""AMPHENOL CORPORATION""","""032095101""","""US0320951017""",null
"""Toast, Inc.""","""888787108""",null,null
…,…,…,…
"""INTEL CORPORATION""","""458140100""",null,null
"""THE SHERWIN-WILLIAMS COMPANY""","""824348106""",null,null
"""THE KROGER CO.""","""501044101""",null,null


In [51]:
normed5.filter(normed5["issuer_name"] != votes["issuer_name"]).select(COMPANY_COLS)

issuer_name,cusip,isin,figi
str,str,str,str
"""Amphenol Corporation""","""032095101""","""US0320951017""",null
"""The Sherwin-Williams Company""","""824348106""","""US8243481061""","""BBG000BSXQV7"""
"""The Sherwin-Williams Company""","""824348106""","""US8243481061""","""BBG000BSXQV7"""
"""Amphenol Corporation""","""032095101""","""US0320951017""",null
"""TOAST, INC.""","""888787108""","""US8887871080""",null
…,…,…,…
"""Intel Corporation""","""458140100""","""US4581401001""","""BBG000C0G1D1"""
"""The Sherwin-Williams Company""","""824348106""","""US8243481061""","""BBG000BSXQV7"""
"""The Kroger Co.""","""501044101""","""US5010441013""","""BBG000BMY992"""


In [42]:
normed5["cusip"].n_unique()

7545

In [14]:
votes.select(COMPANY_COLS)

issuer_name,cusip,isin,figi
str,str,str,str
"""CANADIAN PACIFIC KANSAS CITY L…","""13646K108""","""CA13646K1084""",null
"""AAON, INC.""","""000360206""","""US0003602069""",null
"""AMAZON.COM, INC.""","""023135106""","""US0231351067""",null
"""AMPHENOL CORPORATION""","""032095101""","""US0320951017""",null
"""MICROSOFT CORPORATION""","""594918104""","""US5949181045""",null
…,…,…,…
"""VERALTO CORPORATION""","""92338C103""",null,null
"""ANALOG DEVICES, INC.""","""032654105""",null,null
"""EDITAS MEDICINE, INC.""","""28106W103""",null,null


In [6]:
normed5.select(pl.all().is_null().sum())

issuer_name,cusip,isin,figi
u32,u32,u32,u32
0,11703,8315,514694


In [ ]:
normed4 = normalize_subset4(
    votes,
    connected_subgraphs_postprocessor=partition_companies_graph_where_necessary,
    cols_to_normalize=["issuer_name", "cusip", "isin", "figi"],
)

In [ ]:
pp3 = partial(
    partition_companies_graph_where_necessary,
    # verbose=True,
)


normed3 = normalize_subset3(
    votes,
    connected_subgraphs_postprocessor=pp3,
    cols_to_normalize=["issuer_name", "cusip", "isin", "figi"],
)

In [ ]:
normed3

In [ ]:
normed3.select(COMPANY_COLS.is_null()).sum().sum_horizontal()

In [ ]:
votes.select(COMPANY_COLS.is_null().sum()).sum_horizontal().item()

## Check that removed nodes are repopulated

In [5]:
G = tuple(
    unconsolidated_df_to_subgraphs(
        votes.select(COMPANY_COLS), connected_subgraphs_postprocessor=None
    )
)

In [6]:
sb = extract_specific_name_subgraph(G, "STARBUCKS")

In [ ]:
len(sb.nodes)

In [ ]:
partitioned = partition_companies_graph_where_necessary(sb, verbose=True)

In [ ]:
print(len(partitioned.nodes))
draw_graph(partitioned, 5)

In [10]:
G = tuple(
    unconsolidated_df_to_subgraphs(
        votes.select(COMPANY_COLS), connected_subgraphs_postprocessor=None
    )
)

In [ ]:
list(G[0].nodes.data())[0]

In [ ]:
partition_via_louvain(G[4], verbose=True)
# draw_graph(G[4], 5)

In [ ]:
from record_consolidation.df_consolidations import normalize_subset

pp = partial(
    partition_companies_graph_where_necessary,
    # verbose=True,
)

atomic = extract_normalized_atomic(
    votes.select(COMPANY_COLS), connected_subgraphs_postprocessor=pp
)
normed = normalize_subset(
    votes,
    connected_subgraphs_postprocessor=pp,
    cols_to_normalize=["issuer_name", "cusip", "isin", "figi"],
)

In [ ]:
votes.select(pl.all().is_null().sum())

In [ ]:
normed.select(
    pl.all().is_null().sum()
)  # .filter(pl.col("issuer_name") == pl.lit("AAON, INC."))

In [ ]:
atomic.select(pl.all().is_null().sum())

# ISSUE

## Problem
Nulls are being propagated in the 

## Diagnosis
(Best guess): When nodes are cut wholesale out of the graph - rather than even just having all edges cut - they don't end up in the canonical mapping, and so become null...

In [ ]:
removed_issuers = (
    votes.filter(normed["issuer_name"].is_null())
    .filter(pl.col("issuer_name").is_not_null())["issuer_name"]
    .unique()
    .to_list()
)
removed_issuers

In [ ]:
emerson = extract_specific_name_subgraph(G, "emerson electric co.")
draw_graph(emerson, 5)

In [ ]:
emerson_partitioned = partition_companies_graph_where_necessary(
    emerson, verbose=True, verbose_within_partitioning_algs=True
)

In [ ]:
for issuer in removed_issuers:
    try:
        draw_graph(extract_specific_name_subgraph(G, issuer), size=5)
    except:
        print(issuer)

In [ ]:
votes.select(COMPANY_COLS).filter(
    pl.col("issuer_name") == pl.lit("Northrup Grumman Corporation ")
)

In [ ]:
removed_issuers

In [ ]:
atomized_subset = extract_normalized_atomic(
    votes.select(COMPANY_COLS), connected_subgraphs_postprocessor=pp
)

In [ ]:
s = "walmart"
atomized_subset.filter(pl.col("issuer_name").str.contains(f"(?i){s}"))

In [ ]:
atomized_subset.filter(pl.col("issuer_name").is_in(removed_issuers))

In [26]:
normed_comps = normed.select(COMPANY_COLS)

In [ ]:
normed_comps.filter(pl.col("issuer_name").is_null())

In [ ]:
normed3

In [ ]:
votes

In [ ]:
null_issuers = normed_comps["issuer_name"].is_null()


display(votes.select(COMPANY_COLS).filter(null_issuers).head())  # .sort(
#     pl.all()
# ).unique(maintain_order=True).to_pandas()
display(normed_comps.filter(null_issuers).head())

In [ ]:
atomized_subset.filter(cusip="291011104")

In [ ]:
normed.select(pl.all().is_null().sum())